# Create Multiple New Member Profiles

To use this, open the Google Sheet entitled "PDSG Signup Form (Responses)" and click `File > Download as > Comma-separated values (.csv, current sheet)`. Either copy that file to this directory and rename it as `pdsg_signup.csv` or change `CSV_PATH` below to point to the file

In [69]:
CSV_PATH = 'pdsg_signup.csv'

After that just run the remaining cells, and the script will automatically generate member profiles for every person who signed up, said they wanted to work on team projects, AND don't already have a profile.

In [70]:
from datetime import datetime
import pandas as pd
from glob import glob
from os.path import basename
from new_member import new_member

In [71]:
def get_time_slugs(timestamps):
    """
    Returns a pandas Series of time slugs formatted like '2017-01-02'
    given a pandas Series of timestamps formatted like '1/2/2016 16:20:00'
    """
    time_df = pd.DataFrame()
    time_df['month'], time_df['day'], time_df['year'] = timestamps.str.replace(r'\s.*', '').str.split('/').str
    time_slugs = pd.to_datetime(time_df).dt.strftime("%Y-%m-%d")
    return time_slugs

In [72]:
member_files = glob('members/_posts/*')
member_slugs = [basename(f)[11:-3] for f in member_files]
print('Slugs associated with all current members:')
member_slugs

Slugs associated with all current members:


['ben-lindsay', 'jennifer-hwang', 'rishabh-gupta', 'mayank-darbari']

In [73]:
col_names = ['Timestamp', 'Email', 'Name', 'School', 'Degree', 'Program', 'Grad Year',
             'Python', 'R', 'Mailing List', 'Team Projects', 'Comments']
df = pd.read_csv(CSV_PATH, dtype=str)
df = df.fillna('')
df.columns = col_names
print("Original Dataframe:")
df.head()

Original Dataframe:


,Timestamp,Email,Name,School,Degree,Program,Grad Year,Python,R,Mailing List,Team Projects,Comments
0,12/14/2016 19:23:37,adedotun@wharton.upenn.edu,Adedotun Adejare,Wharton,Bachelors,,,2,1,Yes,Not Sure,
1,12/23/2016 22:42:22,test@example.com,Jimmy Fallon,SEAS,JD,CIS,2019,5,1,Yes,Yes,


In [74]:
df = df[df.iloc[:,-2] == 'Yes']
df['Timestamp'] = get_time_slugs(df['Timestamp'])
df['Email'] = df['Email'].str.replace(u'@', ' at ')
df['Name Slug'] = df['Name'].str.lower().str.replace(' ', '-')
df['Deg String'] = df['Degree'].str.cat([df['Program'], df['Grad Year']], sep=' ')
df = df[['Timestamp', 'Email', 'Name', 'Name Slug', 'Deg String']]
print('Dataframe of people interested in team projects:')
df.head()

Dataframe of people interested in team projects:


,Timestamp,Email,Name,Name Slug,Deg String
1,2016-12-23,test at example.com,Jimmy Fallon,jimmy-fallon,JD CIS 2019


In [75]:
df = df[~df['Name Slug'].isin(member_slugs)]
print("Dataframe of people interested in team projects who don't already have a profile:")
df.head()

Dataframe of people interested in team projects who don't already have a profile:


,Timestamp,Email,Name,Name Slug,Deg String
1,2016-12-23,test at example.com,Jimmy Fallon,jimmy-fallon,JD CIS 2019


In [76]:
row = None
for index, row in df.iterrows():
    new_member(row['Name'], deg_string=row['Deg String'], date_slug=row['Timestamp'], email=row['Email'])
if row is None:
    print("There are no new member profiles to create.")

File created -> members/_posts/2016-12-23-jimmy-fallon.md
Edit this file and add square profile picture at images/members/jimmy-fallon.jpg
